In [1]:
from glob import glob
import numpy as np
import csv
import pandas as pd
from astropy.table import Table
from astropy.io import fits
import os

In [2]:
#define function with inputs field, ID, subfield, zphot; will add more as more data is gathered
def Magnitude(flux):
    #note: had to change to abs(flux) to avoid error, not sure if this is okay
    return -2.5*np.log10(abs(flux)) + 25

In [3]:
def logmass(mass):
    return round(np.log10(mass), 6)

In [23]:
def Bio_gen(field, ID, subfield, catalog, eazy_catalog, griz_catalog):
    # field - eithxer North or South
    # ID - id number of galaxy
    # subfield - ie GN1 or ERSPRIME
    # zphot - photmetric redshift
    
    text1 ='<html>\n\
    <title>{0} {1}</title>\n\
    <meta charset="UTF-8" /><meta name="viewport" content="width=device-width, initial-scale=1" />\n\
    <style>\n'.format(subfield, ID)
    
    text2 = 'table.table1 {font-family: arial;border-collapse: collapse;width: 780px;}\n\
        table.table1 td, table.table1 th {border: 1px solid #ffffff;text-align: center;padding: 7px;}\n\
    * {box-sizing: border-box;}\n\
table.table2 {font-family: arial;border-collapse: collapse;width: 900px;}\n\
    table.table2 td, table.table2 th {border: none;text-align: center;padding: 7px;}\n\
    * {box-sizing: border-box;}\n\
     img.eazy {width: 100%;}\n\
    img.grizli {width: 100%;}\n\
    img.twod {width: 100%;}\n\
    img.line {width: 100%;}\n\
    </style>\n\
    <table align="center" class="table1">\n\
    <tr>\n\
    <th>Field</th>\n\
    <th>ID</th>\n\
    <th>z<sub>phot</sub></th>\n\
    <th>z<sub>grism</sub></th>\n\
    <th>m<sub>F098M</sub></th>\n\
    <th>m<sub>F105W</sub></th>\n\
    <th>m<sub>F125W</sub></th>\n\
    <th>log(M<sub>*</sub>/M<sub>&#9737</sub>)</th>\n\
    <th>ra (deg)</th>\n\
    <th>dec (deg)</th></tr>\n\
    <tr>\n'
    
    #{0} - subfield, {1} - ID, {2} - zphot, {7} - log(mass), {8} - RA, {9} - DEC
    text3='<th>{0}</th>\n\
    <th>{1}</th>\n\
    <th>{2:.2f}</th>\n\
    <th>{3:.4f}</th>\n\
    <th>{4}</th>\n\
    <th>{5:.2f}</th>\n\
    <th>{6:.2f}</th>\n\
    <th>{7}</th>\n\
    <th>{8}</th>\n\
    <th>{9}</th>\n\
    </tr>\n\
    </table>\n\
    <body bgcolor="#000000" text="#ffffff">\n\
    <p> <br /> </p>\n\
    <table align="center" class="table2">\n\
    <tr>\n\
    <th><p> EAZY <br /> P(z) <br /> SED </p></th>\n\
    <th><img class="eazy" src="../../../eazy_plots/{0}_{1}_EZ.png"></th>\n\
    <tr>\n\
    <th><p>Grizli <br /> P(z) <br /> SED <br /> </p></th>\n\
    <th><img class="grizli" src="../../../all_png/{0}_{1}.full.png"></th>\n\
    <tr>\n\
    <th><p> Grizli <br /> 2D Spectra </p></th>\n\
    <th><img class="twod" src="../../../all_png/{0}_{1}.stack.png"></th>\n\
    <tr>\n\
    <th><p> Grizli <br /> Line Map </p></th>\n\
    <th><img class="line" src="../../../all_png/{0}_{1}.line.png"></th>\n\
    </tr>\n\
    </table>\n\
    </p>\n\
    </body>\n\
    </html>\n'.format(subfield, ID, 
                      eazy_catalog.query('id == {0}'.format(int(ID))).z_phot.values[0],
                      griz_catalog.query('ID == {0}'.format(int(ID))).z_50.values[0],'-',
                      Magnitude(catalog.query('id == {0}'.format(int(ID))).f_F105W.values[0]),
                      Magnitude(catalog.query('id == {0}'.format(int(ID))).f_F125W.values[0]),
                      logmass(eazy_catalog.query('id == {0}'.format(int(ID))).mass.values[0]),
                      catalog.query('id == {0}'.format(int(ID))).ra.values[0],
                      catalog.query('id == {0}'.format(int(ID))).dec.values[0])

    #return everything as an array (so np.savetxt can read it)
    return [text1 + text2 + text3]

In [5]:
#read tables from different catalogs
gsd_cat = Table.read('../catalogs/goodss_3dhst.v4.4.cat', format = 'ascii').to_pandas()
gsd_eazy = Table.read(fits.open('../catalogs/goodss_3dhst.v4.4.zout.fits'), format = 'fits').to_pandas()

gnd_cat = Table.read('../catalogs/goodsn_3dhst.v4.4.cat', format = 'ascii').to_pandas()
gnd_eazy = Table.read(fits.open('../catalogs/goodsn_3dhst.v4.4.zout.fits'), format = 'fits').to_pandas()

In [6]:
field_names = ['GN1', 'GN2', 'GN3', 'GN4', 'GN5', 'GN7', 'ERSPRIME', 'GS1', 'GS2', 'GS3', 'GS4', 'GS5']

In [138]:
for i in field_names:
    
    print('Processing '+i+'...')
    
    griz_catalogs = Table.read(fits.open('../grizli_v2.1_cats/' + i + '_lines_grizli.fits'), format = 'fits').to_pandas()
    ids=griz_catalogs['ID']
    
    if 'N' in i:
        F = 'North'
        C = gnd_cat
        E = gnd_eazy
    else:
        F = 'South'
        C = gsd_cat
        E = gsd_eazy
    
    for ii in ids:
        #print(F,i,ii)
        bio_page=Bio_gen(F, ii, i, C, E, griz_catalogs) 
        np.savetxt('/Users/rosaliaobrien/research/bio_pages/{0}_{1}.html'.format(i,ii),bio_page,fmt='%s')

Processing GN1...


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log10
  


Processing GN2...
Processing GN3...
Processing GN4...
Processing GN5...
Processing GN7...
Processing ERSPRIME...
Processing GS1...
Processing GS2...
Processing GS3...
Processing GS4...
Processing GS5...
